In [1]:
def get_time(minute_after_midnight):
    # hh:mm
    hh = minute_after_midnight // 60
    mm = minute_after_midnight % 60
    return str(hh) + ":" + str(mm).zfill(2)


In [2]:
import import_ipynb
from dijkstra import nodes, arc_matrix, get_shortest_path, format_path
from tsp_costruttivo import build_tsp_path

importing Jupyter notebook from dijkstra.ipynb
('1', '2', '3', '4', '5n', '5s', '6', '7', '8', '9', '10', '10b', '11n', '11s', '12', '13', '14', '15', '16', '17', '19', '20', '20b', '21', '22', '23', '24', '25/1', '25/2', '26', '31', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22')
[[inf inf  7. ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [ 7. inf inf ... inf inf inf]
 ...
 [inf inf inf ... inf  4. inf]
 [inf inf inf ...  4. inf inf]
 [inf inf inf ... inf inf inf]]
X1 -> X2 -> 9 -> 7
9.0
2: [2, 36, 35, 35, 33, 33, 32, 47, 32, 7, 37, 42, 43, 44, 45, 41, 40, 40, 17, 46, 50, 52, 52, -1, 49, 48, 47, 51, 27, 50, 25, 32, 9, 9, 33, 34, 34, 9, 37, 38, 39, 16, 39, 38, 45, 46, 47, 48, 49, 50, 51, 21, 23]
1: [55.0, 55.0, 48.0, 47.0, 45.0, 44.0, 40.0, 35.0, 40.0, 37.0, 42.0, 48.0, 45.0, 39.0, 38.0, 51.0, 47.0, 47.0, 50.0, 35.0, 22.0, 8.0, 3.0, 0, 26.0, 31.0, 32.0, 20.0, 22.0, 22.0, 33.0, 44.0, 39.0

In [3]:
import random
import numpy as np

random.seed(0)

MINUTES_IN_A_DAY = 24*60
MINUTES_PER_WORK_CYCLE = 120

distribuzione_tamponi_al_minuto = np.array([random.randint(0, 10) for _ in range(MINUTES_IN_A_DAY)])

distribuzione_reparti = np.array([random.random() for _ in range(len(nodes))])
junction_nodes_start_index = nodes.index('X1')
distribuzione_reparti[junction_nodes_start_index:] = 0
# distribuzione_reparti /= distribuzione_reparti.sum()
# print(distribuzione_reparti)

print(distribuzione_tamponi_al_minuto)
print(distribuzione_reparti)

[6 6 0 ... 3 0 5]
[0.5290949  0.94135742 0.68025796 0.630908   0.62781515 0.49698971
 0.73091927 0.24919444 0.89175426 0.27447266 0.94494501 0.92649671
 0.07792452 0.4481797  0.74403628 0.44965407 0.50889902 0.80682394
 0.70499216 0.95800422 0.16448599 0.92355929 0.92798625 0.63474894
 0.94039083 0.25268559 0.88178728 0.77347929 0.609689   0.09062924
 0.03013435 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]


In [4]:
## funzioni di aggiornamento dei tamponi
def aggiornamento_tamponi(tamponi_per_reparto, current_minute):
    return aggiornamento_tamponi_semplice(tamponi_per_reparto, current_minute)

# ad-hoc
def aggiornamento_tamponi_semplice(tamponi_per_reparto, current_minute):
    if(current_minute==1):
        reparto = nodes.index('22')
        tamponi_per_reparto[reparto] += 10
        print("Arrivati 10 tamponi al reparto 22")
        
    return tamponi_per_reparto

# tramite istanza giornaliera
def aggiornamento_tamponi_istanza(tamponi_per_reparto, current_minute):
    # DA IMPLEMENTARE
    return tamponi_per_reparto

# tramite distribuzione statistica
def aggiornamento_tamponi_statistica(tamponi_per_reparto, current_minute):
    nuovi_tamponi_per_reparto = (distribuzione_tamponi_al_minuto[current_minute] * distribuzione_reparti).astype(int)
    tamponi_per_reparto += nuovi_tamponi_per_reparto

    return tamponi_per_reparto


In [5]:
## funzioni di pianificazione in caso di assenza di tamponi
def pianifica_no_tamponi(tamponi_per_reparto, start, current_minute):
    # return pianifica_goto_lab(tamponi_per_reparto, start, current_minute)
    pianifica_rimani_fermo(tamponi_per_reparto, start, current_minute)

# nessun piano
def pianifica_rimani_fermo(tamponi_per_reparto, start, current_minute):
    return None

# ritorno al laboratorio
LAB = nodes.index('9')
def pianifica_goto_lab(tamponi_per_reparto, start, current_minute):
    end = LAB
    
    (path, distance) = get_shortest_path(arc_matrix, start, end)
    
    path = path[1:]
    return path


In [6]:
## funzioni di pianificazione (lista di nodi nell'ordine in cui si vogliono visitare)
def pianifica(tamponi_per_reparto, current_position, current_minute, tamponi_trasportati):
    # return pianifica_gotomax(tamponi_per_reparto, current_position, current_minute)
    
    if(np.sum(tamponi_per_reparto)==0):
        return pianifica_no_tamponi(tamponi_per_reparto, current_position, current_minute)
    
    current_work_cycle_minute = current_minute % MINUTES_PER_WORK_CYCLE
    residual_time = MINUTES_PER_WORK_CYCLE - current_work_cycle_minute
    path = build_tsp_path(arc_matrix, tamponi_per_reparto, residual_time, tamponi_trasportati, current_position, LAB)
    return path

# si va al reparto con più tamponi da raccogliere, il piano è calcolato con Dijkstra
def pianifica_gotomax(tamponi_per_reparto, start, current_minute):
    end = np.argmax(tamponi_per_reparto)
    
    if tamponi_per_reparto[end] == 0: # non ci sono tamponi da raccogliere
        return pianifica_no_tamponi(tamponi_per_reparto, start, current_minute)

    (path, distance) = get_shortest_path(arc_matrix, start, end)

    path = path[1:]
    return path


In [7]:
## funzioni di valutazione dei piani
def get_score(tamponi_per_reparto, current_position, current_minute, piano):
    return get_score_density(tamponi_per_reparto, current_position, current_minute, piano)

# score = somma dei tamponi da raccogliere diviso il tempo di percorrenza (tamponi/minuto)
def get_score_density(tamponi_per_reparto, current_position, current_minute, piano):
    if piano in (None, []):
        return 0

    score = sum(tamponi_per_reparto[p] for p in piano)
    _, distanza = get_shortest_path(arc_matrix, current_position, piano[-1])
    score /= distanza
    return score

In [8]:
def reconsider(tamponi_per_reparto, current_position, current_minute, piano, tamponi_trasportati):
    score = get_score(tamponi_per_reparto, current_position, current_minute, piano)
    
    piano_alternativo = pianifica(tamponi_per_reparto, current_position, current_minute, tamponi_trasportati)
    score_alternativo = get_score(tamponi_per_reparto, current_position, current_minute, piano_alternativo)
    
    return score_alternativo > score, piano_alternativo

In [9]:
def move_toward(current_position, next_position, moving_time):
    distance = int(get_shortest_path(arc_matrix, current_position, next_position)[1])
    moving_time = min(moving_time+1, distance)
    print(f"Moving from {nodes[current_position]} to {nodes[next_position]} ... {moving_time}/{distance}")
    return distance, moving_time

In [10]:
def get_status(current_minute, current_position, tamponi_trasportati):
    return f"Ora {get_time(current_minute)} Current position: {nodes[current_position]} Tamponi trasporati: {tamponi_trasportati}"

In [11]:
tamponi_per_reparto = np.zeros(len(nodes)).astype(int)
current_position = nodes.index('X1')
current_minute = 0
moving_time = 0
tamponi_trasportati = 0

while current_minute < MINUTES_IN_A_DAY:
    print(get_status(current_minute, current_position, tamponi_trasportati))
        
    tamponi_per_reparto = aggiornamento_tamponi(tamponi_per_reparto, current_minute)
    # print(tamponi_per_reparto)

    piano = pianifica(tamponi_per_reparto, current_position, current_minute, tamponi_trasportati)
    print(f"Piano: {format_path(piano)}")

    while piano not in [[], None]:
        next_position = piano[0]
        distance, moving_time = move_toward(current_position, next_position, moving_time)
        current_minute += 1
        
        if distance == moving_time: # arrivato a destinazione
            print(f"Arrivato al nodo {nodes[next_position]}")
            current_position = next_position
            piano = piano[1:]
            moving_time = 0
            
            if tamponi_per_reparto[current_position] > 0:
                print(f"Tamponi raccolti: {tamponi_per_reparto[current_position]}")
                tamponi_trasportati += tamponi_per_reparto[current_position]
                tamponi_per_reparto[current_position] = 0
        
        print()
        print(get_status(current_minute, current_position, tamponi_trasportati))
        
        tamponi_per_reparto = aggiornamento_tamponi(tamponi_per_reparto, current_minute)
        # print(tamponi_per_reparto)

        rif, piano_alternativo = reconsider(tamponi_per_reparto, current_position, current_minute, piano, tamponi_trasportati)
        if rif:
            print(f"Riconsidero il piano {format_path(piano)} con il piano alternativo {format_path(piano_alternativo)}")
            piano = piano_alternativo
        
        print(f"Piano: {format_path(piano)}")
        
        ## TESTING
        if current_minute > 30:
            break
        ##
    
    ## TESTING
    if current_minute > 30:
        break
    ##

    current_minute += 1
    print()

Ora 0:00 Current position: X1 Tamponi trasporati: 0
Piano: No path

Ora 0:01 Current position: X1 Tamponi trasporati: 0
Arrivati 10 tamponi al reparto 22
Piano: 22 -> 9
Moving from X1 to 22 ... 1/22

Ora 0:02 Current position: X1 Tamponi trasporati: 0
Piano: 22 -> 9
Moving from X1 to 22 ... 2/22

Ora 0:03 Current position: X1 Tamponi trasporati: 0
Piano: 22 -> 9
Moving from X1 to 22 ... 3/22

Ora 0:04 Current position: X1 Tamponi trasporati: 0
Piano: 22 -> 9
Moving from X1 to 22 ... 4/22

Ora 0:05 Current position: X1 Tamponi trasporati: 0
Piano: 22 -> 9
Moving from X1 to 22 ... 5/22

Ora 0:06 Current position: X1 Tamponi trasporati: 0
Piano: 22 -> 9
Moving from X1 to 22 ... 6/22

Ora 0:07 Current position: X1 Tamponi trasporati: 0
Piano: 22 -> 9
Moving from X1 to 22 ... 7/22

Ora 0:08 Current position: X1 Tamponi trasporati: 0
Piano: 22 -> 9
Moving from X1 to 22 ... 8/22

Ora 0:09 Current position: X1 Tamponi trasporati: 0
Piano: 22 -> 9
Moving from X1 to 22 ... 9/22

Ora 0:10 Current